# Hyperparameter Search

In [1]:
# Import classes for tokenization and model training
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

# Import DatasetDict which will help us prepare our own dataset for use in training and evaulating machine learning models
from datasets import DatasetDict

# Import function to be used as loss function
from sklearn.metrics import mean_squared_error

# Import library to track our training runs and change settings
import wandb

# Replace the variables below with your own: name, project name, and project directory
%env WANDB_ENTITY = langdon
%env WANDB_PROJECT = ellipse
%env WANDB_DIR = /home/jovyan/active-projects/ellipse-methods-showcase/bin

score_to_predict = 'Grammar'
model_names = ['bert-base-uncased', 'xlm-roberta-base', 'microsoft/deberta-base']

env: WANDB_ENTITY=langdon
env: WANDB_PROJECT=ellipse
env: WANDB_DIR=/home/jovyan/active-projects/ellipse-methods-showcase/bin


In [2]:
def get_datadict(score_to_predict):
    ''' Selects a target score that the model should predict and renames that score to 'label'.
    Removes other columns from the dataset. The other columns are not needed for training.
    '''
    
    # These columns will be removed from the dataset
    scores = {
        'Overall',
        'Cohesion',
        'Syntax',
        'Vocabulary',
        'Phraseology',
        'Grammar',
        'Conventions'
    }
    
    columns_to_remove = scores.symmetric_difference([score_to_predict])
    
    # Load the DatasetDict object we created in the previous notebook. 
    # We will be removing the columns that we defined above, and renaming the target column (=score_to_predict) into 'label'
    dd = (DatasetDict
          .load_from_disk('../data/ellipse.hf')
          .remove_columns(columns_to_remove)
          .rename_column(score_to_predict, 'label') # Huggingface will look for a column that contains the string 'label' to calculate metrics.
         )
    
    return dd

# Load dataset using the function
datadict = get_datadict(score_to_predict)

In [3]:
# Create a function that will help us evaluate the model's performance by calculating the mean squared error of the model's predictions
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    mse = mean_squared_error(labels, preds)

    return {'mse': mse}

In [4]:
def sweep(score_to_predict, model_name):

    tokenizer = AutoTokenizer.from_pretrained(model_name)   
    
    def model_init(trial):
        return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)

    def tokenize_inputs(example):
        return tokenizer(example['text'], max_length=512, truncation=True)

    sweep_config = {
        'name': f'{score_to_predict}-{model_name}-optimization',
        'method': 'bayes',
        'metric': {  # we want to "minimize" the mean squared error.
            'name': 'eval/mse',
            'goal': 'minimize'
        },
        'parameters': {
            'batch_size': {'value': 16},
            'learning_rate': {'values': [1e-5, 2e-5, 3e-5, 4e-5]},
            'warmup_steps': {'values': [50, 100, 500, 1000]},
            'weight_decay': {'values': [0.01, 0.1]},
            'epochs': {'values': [2, 3, 4, 5]},
        },
        'run_cap': 12,
    }

    dd_tokenized = datadict.map(tokenize_inputs, remove_columns=['text_id', 'text'])

    def train():
        with wandb.init():
            # set sweep configuration
            config = wandb.config
    
            # Customize the trainer
            training_args = TrainingArguments(
                output_dir = f'../bin/{score_to_predict}-{model_name}', 
                optim = 'adamw_torch', # Specify your optimizer
                logging_dir = f'../logs/{score_to_predict}-{model_name}', 
                load_best_model_at_end = True,
                metric_for_best_model = 'mse', # We will be using mean squared error to evaluate model performance
                evaluation_strategy='epoch', # Evaluate model performance at the end of each epoch
                save_strategy='epoch',
                save_total_limit=1,
                greater_is_better = False,
                log_level = 'error',
                disable_tqdm = False,
                report_to='wandb',
                # The hyper parameters we are tuning (umber of epochs, learning rate, and batch size) are called in from the configuration dictionary
                warmup_steps=config.warmup_steps,
                weight_decay=config.weight_decay,
                num_train_epochs=config.epochs, 
                learning_rate=config.learning_rate,
                per_device_train_batch_size=config.batch_size,
                per_device_eval_batch_size=16,
            )
    
            # Initialize the trainer
            trainer = Trainer(
                model=None, # this is to emphasize that we are not passing the model directly
                args=training_args,
                train_dataset=dd_tokenized['train'],
                eval_dataset=dd_tokenized['dev'],
                compute_metrics=compute_metrics,
                tokenizer=tokenizer,
                model_init=model_init, # we pass a function that initializes the model afresh at the start of each trial
            )
    
    
            # Start training loop
            trainer.train()

    sweep_id = wandb.sweep(sweep_config)
    wandb.agent(sweep_id, train)

In [ ]:
for model_name in model_names:
    sweep(score_to_predict, model_name)

Map:   0%|          | 0/4537 [00:00<?, ? examples/s]

Map:   0%|          | 0/972 [00:00<?, ? examples/s]

Map:   0%|          | 0/973 [00:00<?, ? examples/s]

Create sweep with ID: xhklmi1i
Sweep URL: https://wandb.ai/langdon/ellipse/sweeps/xhklmi1i


wandb: Agent Starting Run: dnrdrl61 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 2
wandb: 	learning_rate: 2e-05
wandb: 	warmup_steps: 100
wandb: 	weight_decay: 0.01
wandb: Currently logged in as: langdon. Use `wandb login --relogin` to force relogin


...

Map:   0%|          | 0/973 [00:00<?, ? examples/s]

Create sweep with ID: ouo8swcb
Sweep URL: https://wandb.ai/langdon/ellipse/sweeps/ouo8swcb


wandb: Agent Starting Run: 4fgapib4 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 2
wandb: 	learning_rate: 2e-05
wandb: 	warmup_steps: 100
wandb: 	weight_decay: 0.1


wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_steps' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss
